<a href="https://colab.research.google.com/github/moon-2000/VID-Trans-ReID/blob/main/VID_Train_MARS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cloning VID Repository

In [ ]:
#!git clone https://github.com/AishahAADU/VID-Trans-ReID.git

Cloning into 'VID-Trans-ReID'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 122 (delta 40), reused 25 (delta 25), pack-reused 70
Receiving objects: 100% (122/122), 74.01 KiB | 12.33 MiB/s, done.
Resolving deltas: 100% (59/59), done.


### Mounting Drive

In [1]:
# mounting the drive (connecting colab to the drive folder of the project)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp  -r /content/drive/MyDrive/Colab_Notebooks/VID/VID-Trans-ReID   /content/

cp: cannot stat '/content/drive/MyDrive/Colab_Notebooks/VID/VID-Trans-ReID': No such file or directory


### Installing VID repository and MARS dataset on Drive

In [ ]:
# to save the folder after cloning on drive (make a copy and save it on drive)
#!cp -r VID-Trans-ReID drive/MyDrive/Colab_Notebooks/VID

In [ ]:
# to check the home directory content
!ls

drive  sample_data  VID-Trans-ReID


In [ ]:
# to copy the zipeed dataset file from drive to colab environment
%cp /content/drive/MyDrive/Colab_Notebooks/VID/MARS/MARS.zip  /content/

In [ ]:
!sleep 10
%ls

sample_data/


In [ ]:
 # to unzip MARS dataset
!unzip /content/MARS.zip -d /content/MARS

In [ ]:
%%shell
mv /content/MARS/bbox_test /content/MARS/bbox_test2
mv /content/MARS/bbox_test2/bbox_test /content/MARS/
rm -r /content/MARS/bbox_test2
mv /content/MARS/bbox_train /content/MARS/bbox_train2
mv /content/MARS/bbox_train2/bbox_train /content/MARS/
rm -r /content/MARS/bbox_train2

In [ ]:
%cp /content/drive/MyDrive/Colab_Notebooks/VID/MARS/info.zip  /content/

In [ ]:
!unzip info.zip -d /content/MARS

unzip:  cannot find or open info.zip, info.zip.zip or info.zip.ZIP.


In [ ]:
# open the VID repo
%cd /content/VID-Trans-ReID

/content/VID-Trans-ReID


In [ ]:
 # check the repo content (to be sure you are in the directory)
!ls

checkpoints    loss	    README.md	      utility.py	  VID_Trans_ReID.py
Dataloader.py  Loss_fun.py  requirements.txt  VID_Test.py	  visualize_results.py
Datasets       __pycache__  results	      VID_Trans_model.py  vit_ID.py


In [ ]:
# to download MARS dataset - ONLY 5kB are downloaded - NOT WOrking
# !wget -O MARS.zip "https://www.kaggle.com/datasets/twoboysandhats/mars-motion-analysis-and-reidentification-set/download?datasetVersionNumber=1"

In [ ]:
# to move the dataset into drive
# %mv MARS /content/drive/MyDrive/Colab_Notebooks/VID

### Installing Requirements

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00


### Train and Evaluate on MARS Dataset

In [ ]:
# Remember to change  in vit_ID.py line 9
#from torch._six import container_abcs            to    #import collections.abc as container_abcs

# in VID_Trans_ReID.py

#parser.add_argument("--ViT_path", default="", help="The name of the pretrained model", type=str)


#and change line 163 to

#model = VID_Trans( num_classes=num_classes, camera_num=camera_num,pretrainpath=args.ViT_path)

# and line 90 -- from
    # imgs = Variable(imgs)
    # to   with torch.no_grad():
            # imgs = Variable(imgs)


# in dataloader, change
# root = '/content/MARS'

# line 67 in Dataloader.py  -- change the batch_size = 16 instead of batch_size = 64

In [ ]:
!pip install torch_ema

In [ ]:
%mkdir weights
%cp /content/drive/MyDrive/Colab_Notebooks/VID/jx_vit_base_p16_224-80ecf9dd.pth ./weights/

cp: cannot stat '/content/drive/MyDrive/Colab_Notebooks/VID/jx_vit_base_p16_224-80ecf9dd.pth': No such file or directory


In [ ]:
%cd /content/VID-Trans-ReID/

/content/VID-Trans-ReID


In [ ]:
# on 10 epochs, eval after 10 epochs - 26.10
!python -u VID_Trans_ReID.py --Dataset_name 'Mars'  --ViT_path  '/content/VID-Trans-ReID/weights/jx_vit_base_p16_224-80ecf9dd.pth'

=> MARS loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   625 |     8298
  query    |   626 |     1980
  gallery  |   622 |     9330
  ------------------------------
  total    |  1251 |    19608
  number of images per tracklet: 2 ~ 920, average 59.5
  ------------------------------
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
using stride: [16, 16], and patch number is num_y16 * num_x8
Resized position embedding from size:torch.Size([1, 197, 768]) to size: torch.Size([1, 12

In [ ]:
# train ViT base-model on MARS dataset - on 120 epochs - incomplete
!python -u VID_Trans_ReID.py --Dataset_name 'Mars'  --ViT_path  '/content/VID-Trans-ReID/weights/jx_vit_base_p16_224-80ecf9dd.pth'

=> MARS loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # tracklets
  ------------------------------
  train    |   625 |     8298
  query    |   626 |     1980
  gallery  |   622 |     9330
  ------------------------------
  total    |  1251 |    19608
  number of images per tracklet: 2 ~ 920, average 59.5
  ------------------------------
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
using stride: [16, 16], and patch number is num_y16 * num_x8
Resized position embedding from size:torch.Size([1, 197, 768]) to size: torch.Size([1, 12